<a href="https://colab.research.google.com/github/MathBorgess/discrete-to-binary-regression/blob/main/titanic_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
titanic_dataset = pd.read_csv('Titanic-Dataset.csv').drop(columns=['PassengerId','Ticket', 'Cabin'])
titanic_dataset = titanic_dataset[titanic_dataset['Embarked'].notna()]
#Trying to give some meaning to the correlations between data and the dependent variable
titanic_dataset.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [24]:
titanic_dataset.loc[titanic_dataset['Age'].isna() & titanic_dataset['Name'].str.contains('Master'), 'Age'] = 5
titanic_dataset.groupby(['Sex', 'Pclass'])['Age'].agg(['mean', 'count'])

mean  count
Sex    Pclass                  
female 1       34.240964     83
       2       28.722973     74
       3       21.750000    102
male   1       41.281386    101
       2       30.740707     99
       3       26.172840    257

In [30]:
titanic_dataset.loc[(titanic_dataset['Age'].isna()) & (titanic_dataset['Sex'] == 'female') & (titanic_dataset['Pclass'] == 1), 'Age'] = 34
titanic_dataset.loc[(titanic_dataset['Age'].isna()) & (titanic_dataset['Sex'] == 'female') & (titanic_dataset['Pclass'] == 2), 'Age'] = 29
titanic_dataset.loc[(titanic_dataset['Age'].isna()) & (titanic_dataset['Sex'] == 'female') & (titanic_dataset['Pclass'] == 3), 'Age'] = 22

titanic_dataset.loc[(titanic_dataset['Age'].isna()) & (titanic_dataset['Sex'] == 'male') & (titanic_dataset['Pclass'] == 1), 'Age'] = 41
titanic_dataset.loc[(titanic_dataset['Age'].isna()) & (titanic_dataset['Sex'] == 'male') & (titanic_dataset['Pclass'] == 2), 'Age'] = 31
titanic_dataset.loc[(titanic_dataset['Age'].isna()) & (titanic_dataset['Sex'] == 'male') & (titanic_dataset['Pclass'] == 3), 'Age'] = 26

titanic_dataset = titanic_dataset.drop(columns=['Name'])

In [31]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
encoder = OneHotEncoder()
to_binary_values = titanic_dataset[['Pclass', 'SibSp', 'Parch', 'Embarked', 'Sex']]
encoder.fit(to_binary_values)
binary_values = encoder.transform(to_binary_values).toarray()
binary_values = np.array([''.join(row.astype(str)) for row in binary_values.astype(int)])
X_base10 = np.array([int(x, 2) for x in binary_values]).reshape(-1, 1)
continum_titanic_dataset = titanic_dataset.drop(columns=['Pclass', 'SibSp', 'Parch', 'Embarked', 'Sex'])
continum_titanic_dataset['new_axis'] = X_base10
continum_titanic_dataset.corr()

,Survived,Age,Fare,new_axis
Survived,1.000000,-0.071206,0.255290,0.316800
Age,-0.071206,1.000000,0.115138,0.436555
Fare,0.255290,0.115138,1.000000,0.551106
new_axis,0.316800,0.436555,0.551106,1.000000


In [33]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

test = []
for i in range(10):
    X = continum_titanic_dataset[continum_titanic_dataset['Age'].notna()]

    X_train, X_val, y_train, y_val = train_test_split(X.drop(columns=['Survived']), X[['Survived']], test_size=0.2)

    standarize = StandardScaler()
    standarize.fit(X_train)

    X_train_transformed = standarize.transform(X_train)

    svc = LinearSVC()
    svc.fit(X_train,y_train)
    y_pred = svc.predict(X_val)
    test.append(accuracy_score(y_val, y_pred))
test

c:\Users\Diego\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Diego\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Diego\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Diego\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\U

[0.6460674157303371,
 0.6348314606741573,
 0.601123595505618,
 0.398876404494382,
 0.601123595505618,
 0.3258426966292135,
 0.6235955056179775,
 0.6123595505617978,
 0.6741573033707865,
 0.3202247191011236]